# Using OHLC scan endpoint for pulling in daily price tracking for various coins

see : https://syve.readme.io/reference/price_historical_ohlc

Targetting these coins :
- USDT
- WBTC
- stETH

with necessary identity info stored in my simulated secrets manager.

In [1]:
from cryptoTracker.db.utils import init_pg_db , get_pg_db
from cryptoTracker.application.queries.token import (
    upsert_token, validate_token_info, upsert_token_price_history, get_price_history , get_tokens
)


init_pg_db()

initialising postgres db
connecting to winhost:5433


In [2]:
from cryptoTracker.application.extracts.coin_price_history import CoinPriceExtractor


extractor = CoinPriceExtractor()

extractor.extract_history(from_secrets=True,date_start = '2015-01-01' , date_end='2020-06-06')

Upserting 18 records into price_history for token 1
Upserting 19 records into price_history for token 10


KeyError: "['timestamp_open', 'timestamp_close'] not found in axis"

# Simple visualisation using plotly 

In [2]:
import plotly.graph_objects as go
import pandas as pd
from datetime import datetime

df_tokens , tokens = get_tokens()

In [3]:
df_tokens

,id,name,identifier,chain,eth_contract_address
0,1,Tether USD,USDT,Ethereum,0xdAC17F958D2ee523a2206206994597C13D831ec7
1,10,Wrapped Bitcoin,WBTC,Ethereum,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599
2,11,Lido Staked Ether,stEth,Ethereum,0xae7ab96520de3a18e5e111b5eaab095312d7fe84


In [4]:
import plotly.io as pio
pio.renderers.default = 'iframe'

In [ ]:
df_history.sort_values('price_low',ascending=True)

In [ ]:
for token in df_tokens.itertuples():
    df_history = get_price_history(token.id)

    # filtering for last 30D...
    #df_history = df_history[df_history.date_open.gt(pd.to_datetime('2024-06-30')) & df_history.date_close.lt(pd.to_datetime('2024-08-07'))]
    
    fig = go.Figure(data=[go.Candlestick(x=df_history['date_open'],
                open=df_history['price_open'],
                high=df_history['price_high'],
                low=df_history['price_low'],
                close=df_history['price_close'])])
    fig.update_layout(title=f'{token.name} ({token.identifier})')
    with open(f'figures/price_history/{token.name}.png','wb') as file:
        fig.write_image(file)
    fig.show()

In [ ]:
import plotly.express as px
# generating linegraphs using price_open...

for token in df_tokens.itertuples():
    df_history = get_price_history(token.id)
    # filtering for last 30D...
    df_history = df_history[df_history.date_open.gt(pd.to_datetime('2024-06-30')) & df_history.date_close.lt(pd.to_datetime('2024-08-07'))]
    df_history = df_history.sort_values('date_open',ascending=True)
    fig = px.line(df_history, x="date_open", y="price_open", title=f'{token.name} ({token.identifier})')
    fig.show()